# **Especialização em Ciência de Dados - INF/UFRGS e SERPRO**
### Disciplina CD003 - Aprendizado Supervisionado
#### *Profa. Mariana Recamonde-Mendoza (mrmendoza@inf.ufrgs.br)*
<br> 

---
***Observação:*** *Este notebook é disponibilizado aos alunos como complemento às aulas síncronas e aos slides preparados pela professora. Desta forma, os principais conceitos são apresentados no material teórico fornecido. O objetivo deste notebook é reforçar os conceitos e demonstrar questões práticas no uso de diferentes algoritmos e estratégias de Aprendizado de Máquina. O desenvolvimento deste notebook teve o auxílio do mestrando Thomas Fontanari.*


---

<br>

## **Aula 06** - **Tópico: Árvores de Decisão**

<br>

As árvores de decisão são modelos amplamente conhecidos pela sua inerente interpretabilidade, podendo ser aplicados para tarefas de classificação e regressão. A estrutura de uma árvore de decisão é bastante simples, podendo ser transcrita como um conjunto de regras, cada qual definida pelo caminho percorrido do nó raiz a um nó folha. Entretanto, o algoritmo possui a capacidade de modelar fronteiras de decisão bastante complexas. Apesar destas vantagens, o algoritmo apresenta duas desvantagens importantes: é muito suscetível ao fenômeno de *overfitting* (sobreajuste aos dados) e muito sensível a variações nos dados de treinamento. 

Nessa atividade, vamos explorar árvores de decisão a fim de ver exemplos práticos de suas principais vantagens e desvantagens. Também exploraremos técnicas de pré-poda e pós-poda para controlar a complexidade do modelo e reduzir as chances de overfitting.

<br> 

**Objetivo deste notebook**: Explorar o uso de árvores de decisão para tarefas preditivas, compreendendo seu potencial de interpretabilidade. Avaliar a sensibilidade do modelo a variações nos dados e o impacto de estratégias de regularização do modelo, ao restringir a liberdade do algoritmo de modelar a árvore durante o treinamento do modelo ou realizar pós-poda.

<br>

---



##**Um novo preditor para o diagnóstico de Câncer de Mama**

Para esta atividade, vamos retomar o conjunto de dados sobre câncer de mama usado na Aula 02 (KNN). Lembrando: cada instância se refere ao exame de um(a) paciente e os atributos são computados a partir de uma imagem digitalizada de material coletado de uma massa mamária através de uma punção aspirativa por agulha fina (PAAF), descrevendo as características dos núcleos celulares presentes na imagem. O objetivo da tarefa é ser capaz de prever se o tumor é maligno ou benigno a partir destas características.

Um total de dez características foram extraídas para cada núcleo celular:

*   raio (média das distâncias do centro aos pontos do perímetro)
*   textura (desvio padrão dos valores de escala de cinza)
*   perímetro
*   área
*   suavidade (variação local nos comprimentos dos raios)
*   compacidade (perímetro^2 / área - 1,0)
*   concavidade (gravidade das porções côncavas do contorno)
*   pontos côncavos (número de porções côncavas do contorno)
*   simetria
*   dimensão fractal ("aproximação do litoral" - 1)

Para cada característica foram extraídas a média, o erro padrão e o pior (ou maior) valor, resultando em 30 atributos para cada exame. A última coluna, 'diagnosis', contém a classe verdadeira de cada instância, que pode ser M (maligno) ou B (benigno).
 


###Carregando e inspecionando os dados

Além de possuir uma grande quantidade de algoritmos de aprendizado de máquina, a biblioteca [scikit-learn](https://scikit-learn.org/stable/index.html) possui também funções para carregar alguns conjuntos de dados. Neste notebook, vamos exemplificar o uso destas funções, fazendo o carregamento do dataset [Breast Cancer Wisconsin](https://scikit-learn.org/stable/datasets/toy_dataset.html#breast-cancer-dataset). Este dataset é o mesmo utilizado na Aula 02, apenas mudaremos a forma de carregá-los no código.

In [ ]:
import pandas as pd             # biblioteca para análise de dados 
import matplotlib.pyplot as plt # biblioteca para visualização de informações
import seaborn as sns           # biblioteca para visualização de informações
import numpy as np              # biblioteca para operações com arrays multidimensionais
from sklearn.datasets import load_breast_cancer ## conjunto de dados a ser analisado
sns.set()

data = load_breast_cancer() ## carrega os dados de breast cancer
X = data.data  # matriz contendo os atributos
y = data.target  # vetor contendo a classe (0 para maligno e 1 para benigno) de cada instância
feature_names = data.feature_names  # nome de cada atributo
target_names = data.target_names  # nome de cada classe

print(f"Dimensões de X: {X.shape}\n")
print(f"Dimensões de y: {y.shape}\n")
print(f"Nomes dos atributos: {feature_names}\n")
print(f"Nomes das classes: {target_names}")

In [ ]:
## transforma NumPy Array para Pandas DataFrame
data_df = pd.DataFrame(X,columns=feature_names)

## sumariza os atributos numéricos (todos, neste caso)
data_df.describe()

Relembrando o número de exemplos por classe. Neste dataset carregado do scikit-learn, 0 representa maligno e 1 representa benigno (ao contrário do que usamos anteriormente).

In [ ]:
n_malign = np.sum(y == 0)
n_benign = np.sum(y == 1)

print("Número de exemplos malignos: %d" % n_malign)
print("Número de exemplos benignos: %d" % n_benign)


---

### Treinando uma árvore de decisão




Vamos iniciar a atividade treinando uma árvore de decisão seguindo o padrão do método DecisionTreeClassifier, sem especificar critérios de pré-poda ou pós-poda. Faremos uma divisão inicial dos dados em treino e teste para permitir a posterior avaliação do modelo.

Todos os modelos deste notebook serão treinados utilizando o **critério Gini** para divisão de nós, que é o padrão adotado pelo scikit-learn.

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split  # função do scikit-learn que implementa um holdout

## separa os dados em treino e teste, de forma estratificada
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42,stratify=y) ## atenção: não mude o random_state para este exercício

## treina uma árvore de decisão com configurações 'padrão'
dtStd = DecisionTreeClassifier(random_state=0)
dtStd.fit(X_train, y_train)

In [ ]:
import graphviz
from sklearn import tree

dot_data = tree.export_graphviz(dtStd,         
                                out_file=None,
                                feature_names = feature_names,
                                class_names= target_names, 
                                filled=True)

## Plotar a árvore de decisão no notebook
graph = graphviz.Source(dot_data) 
graph

## Para salvar como png, descomente as linhas abaixo
#graph.format = 'png'
#graph.render('DecisionTree1',view = True)


**Responda >>>** *i)* Qual a profundidade da árvore treinada? *ii)* Qual parece ser o atributo mais informativo deste problema, de acordo com a árvore obtida?  

> ***Sua resposta aqui:*** A árvore tem profundidade 8.

**Responda >>>** Todos os 30 atributos disponíveis são testados na árvore? Em caso negativo, mencione quais foram avaliados pelo menos uma vez.

> ***Sua resposta aqui:*** Não. 14 atributos foram avaliados, sendo eles: 1) worst radius, 2) worst concave points, 3) texture error, 4) radius error, 5) worst texture, 6) worst concavity, 7) area error, 8) mean smoothness, 9) worst symmetry, 10) smoothness error, 11) worst area, 12) worst compactness, 13) worst perimeter, 14) ractal dimension error.

**Responda >>>** Qual o menor e maior tamanho de amostras nos nós folhas desta árvore? Você acha que há sinais de *overfitting*?

> ***Sua resposta aqui:*** O menor nó folha tem tamanho 1, e o maior nó folha tem tamanho 236. Há sinais de overfitting no lado esquerdo da árvore.

####Avaliação do desempenho
Vamos avaliar o desempenho da árvore de decisão com os dados de teste. Para o cálculo de precisão e recall, assumimos a classe Tumor Maligno (0) como positiva.

In [ ]:
from sklearn.metrics import confusion_matrix, recall_score, precision_score,accuracy_score,ConfusionMatrixDisplay ## para avaliação dos modelos

y_pred = dtStd.predict(X_test)

cm = confusion_matrix(y_test, y_pred,labels=dtStd.classes_)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=dtStd.classes_)
disp = disp.plot(include_values=True, cmap='Blues', ax=None, xticks_rotation='horizontal')
plt.grid(False)
plt.show()

print('Acurácia: {}'.format(round(accuracy_score(y_test, y_pred),3)))
print('Recall: {}'.format(round(recall_score(y_test, y_pred,pos_label=0),3)))
print('Precisão: {}'.format(round(precision_score(y_test, y_pred,pos_label=0),3)))


### Análise de instabilidade em árvores de decisão
Como estudado em aula, a árvore de decisão é conhecida por ser um classificador cuja fronteira de decisão é muito sensível a variações nos dados usados para treinamento. Assim, diz-se que árvores de decisão são modelos com alta variância. Isto possui consequências na estrutura das árvores treinadas.

O código abaixo treina várias árvores de decisão com diferentes conjuntos de treino obtidos através do método 2-way holdout. Após conclusão da execução da célula de código, observe as árvores geradas. Os arquivos ficam armazenados na pasta 'content' do Google Colab.


In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split  # função do scikit-learn que implementa um holdout


def get_root_node(dt, feature_names):
    feature_idx = dt.tree_.feature[0]
    return feature_names[feature_idx]


n_repeats = 10
root_nodes = []

## variando o seed do holdout, geramos conjuntos de treino e teste um pouco diferentes a cada iteração
for split_random_state in range(0, n_repeats):
  ## Holdout com 20% de dados de teste
  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=split_random_state,stratify=y)

  ## Treinamento da árvore usando os dados de treino
  dt = DecisionTreeClassifier(random_state=0)
  dt.fit(X_train, y_train)

  ## Obtemos o atributo usado na raiz e o salvamos na lista
  root_node = get_root_node(dt, feature_names)
  root_nodes.append(root_node)  

  ## Opcionalmente, visualize a estrutura da árvore treinada:
  dot_data = tree.export_graphviz(dt,         
                                out_file=None,
                                feature_names = feature_names,
                                class_names= target_names, 
                                filled=True)

  ## Plotar a árvore de decisão no notebook
  graph = graphviz.Source(dot_data) 
  graph
  
  ## Para salvar como png, descomente as linhas abaixo
  graph.format = 'png'
  graph.render('DecisionTree '+str(split_random_state),view = True)

## Imprimindo atributos testados no nó raiz ao longo de todas as execuções.
root_nodes

**Responda >>>** Analisando visualmente as árvores de decisão geradas, bem como o resumo dos atributos selecionados como nó raiz, você observou muitas variações entre as estruturas das árvores nas diferentes iterações? Relate de forma breve. Mencione na sua resposta se e como o atributo do nó raiz variou. Em caso de variação, cite os dois mais frequentes.




> ***Sua resposta aqui:*** Sim, a estrutura das árvores variou muito, apesar das alturas serem parecidas. O atributo do nó raiz não teve tanta variação com 3 atributos, sendo os mais frequentes "worst perimeter" com 5 ocorrências e "worst radius" com 3 ocorrências.

####Avaliação do impacto no desempenho
É esperado que mudanças na estrutura da árvore causem variações no desempenho do modelo. Vamos avaliar esta hipótese, observando a acurácia como medida de desempenho (outras medidas poderiam ser usadas, você pode fazer alterações no código para testá-las).

A célula de código abaixo visa executar repetidas vezes o treinamento das árvores de decisão, da mesma forma que no item anterior (entretanto, executa um número maior de vezes).

Faça as inclusões sugeridas de forma a obter a acurácia do modelo em cada execução e então calcule a média, desvio padrão, máximo e mínimo dos valores. 
**Não mude os valores que estão sendo passados para os parâmetros random_state**.



**Experimente >>>** Faça as adições solicitadas no código abaixo para calcular o que se pede.

In [ ]:
n_repeats = 20
accuracies = [] ## para armazenar a lista de acurácia de todas as repetições

## variando o seed do holdout, geramos conjuntos de treino e teste um pouco diferentes a cada iteração
for split_random_state in range(0, n_repeats):
  ## Holdout com 20% de dados de teste
  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=split_random_state,stratify=y)

  ## Treinamento da árvore usando os dados de treino
    ### 
    # Adicione a sua resposta aqui
    ##
  dt = DecisionTreeClassifier(random_state=0)
  dt.fit(X_train, y_train)


  ##  Faça a predição da saída para os dados de teste e avalie a acurácia (accuracy_score())
    ### 
    # Adicione a sua resposta aqui
    ##
  y_pred = dt.predict(X_test)
  #print('Acurácia: {}'.format(round(accuracy_score(y_test, y_pred),3)))


  ##Monte uma lista dos valores obtidos de acurácia (use ".append(valor a ser adicionado)")
    ### 
    # Adicione a sua resposta aqui
    ##
  accuracies.append(round(accuracy_score(y_test, y_pred),3))


## Ao final, calcule a média, desvio padrão, máximo e mínimo das acurácias 
## (pode usar funções do numpy: np.mean, np.std, np.max, np.min)
  ### 
  # Adicione a sua resposta aqui
  ##
print('Média: {}'.format(np.mean(accuracies)))
print('Min: {}'.format(np.min(accuracies)))
print('Max: {}'.format(np.max(accuracies)))
print('Desvio padrão: {}'.format(np.std(accuracies)))


**Responda >>>** O desempenho do modelo de árvore de decisão sofreu alterações entre as múltiplas repetições? Reporte um resumo dos valores encontrados (por exemplo, valor máximo, mínimo e média).

> ***Sua resposta aqui:*** Sim, variou com desvio padrão de 0.262 em torno da média 0.934.

Média: 0.9342499999999999

Min: 0.886

Max: 0.974

Desvio padrão: 0.026216168675075285

####Avaliação do impacto na fronteira de decisão
Vamos observar a fronteira de decisão gerada para diferentes divisões de conjuntos de treino e teste, para averiguar o quanto a mesma pode ser impactada por variações nos dados. 

Para **fins de visualização**, assumimos como atributos preditivos apenas os **dois primeiros de X_train**. Além disso, aplicamos o algoritmo em um subconjunto dos dados para tornar a visualização mais clara.

In [ ]:
## Função para plotar a fronteira de decisão
## Fonte: https://github.com/tirthajyoti/Machine-Learning-with-Python/blob/master/Utilities/ML-Python-utils.py
def plot_decision_boundaries(X, y, model_class, **model_params):
    """
    Function to plot the decision boundaries of a classification model.
    This uses just the first two columns of the data for fitting 
    the model as we need to find the predicted value for every point in 
    scatter plot.
    Arguments:
            X: Feature data as a NumPy-type array.
            y: Label data as a NumPy-type array.
            model_class: A Scikit-learn ML estimator class 
            e.g. GaussianNB (imported from sklearn.naive_bayes) or
            LogisticRegression (imported from sklearn.linear_model)
            **model_params: Model parameters to be passed on to the ML estimator
    
    Typical code example:
            plt.figure()
            plt.title("KNN decision boundary with neighbros: 5",fontsize=16)
            plot_decision_boundaries(X_train,y_train,KNeighborsClassifier,n_neighbors=5)
            plt.show()
    """
    try:
        X = np.array(X)
        y = np.array(y).flatten()
    except:
        print("Coercing input data to NumPy arrays failed")

    # Reduces to the first two columns of data - for a 2D plot!
    reduced_data = X[:, :2]
    # Instantiate the model object
    model = model_class(**model_params)
    # Fits the model with the reduced data
    model.fit(reduced_data, y)

    # Step size of the mesh. Decrease to increase the quality of the VQ.
    h = .02     # point in the mesh [x_min, m_max]x[y_min, y_max].    

    # Plot the decision boundary. For that, we will assign a color to each
    x_min, x_max = reduced_data[:, 0].min() - 0.5, reduced_data[:, 0].max() + 0.5
    y_min, y_max = reduced_data[:, 1].min() - 0.5, reduced_data[:, 1].max() + 0.5
    # Meshgrid creation
    xx, yy = np.meshgrid(np.arange(x_min, x_max, h), np.arange(y_min, y_max, h))

    # Obtain labels for each point in mesh using the model.
    Z = model.predict(np.c_[xx.ravel(), yy.ravel()])    

    x_min, x_max = X[:, 0].min() - 0.5, X[:, 0].max() + 0.5
    y_min, y_max = X[:, 1].min() - 0.5, X[:, 1].max() + 0.5
    xx, yy = np.meshgrid(np.arange(x_min, x_max, 0.01),
                         np.arange(y_min, y_max, 0.01))

    # Predictions to obtain the classification results
    Z = model.predict(np.c_[xx.ravel(), yy.ravel()]).reshape(xx.shape)

    # Plotting
    plt.contourf(xx, yy, Z, alpha=0.2,cmap='viridis')
    g=plt.scatter(X[:, 0], X[:, 1], c=y, alpha=0.6,s=50, edgecolor='k', cmap='viridis' )
    plt.xlabel("Feature-1",fontsize=15)
    plt.ylabel("Feature-2",fontsize=15)
    plt.xticks(fontsize=14)
    plt.yticks(fontsize=14)
    plt.legend(handles=g.legend_elements()[0],labels=('benign','malign'))
    return plt

In [ ]:
n_repeats = 10

for split_random_state in range(0, n_repeats):
  ## Holdout com 20% de dados de teste
  X_train, X_test, y_train, y_test = train_test_split(X[:,:2], y, test_size=0.2, random_state=split_random_state,stratify=y)
  
  plt.figure(figsize=(6, 4))
  ## chama a função para plotar a fronteira de decisão, a qual faz o treinamento
  ## do modelo com os dados informados
  plot_decision_boundaries(X_train[:50,:],y_train[:50],DecisionTreeClassifier, random_state=0)

**Responda >>>** A análise das fronteira de decisão sugere a ocorrência de overfitting? Justifique

> ***Sua resposta aqui:*** Sim, além da área que delimita uma grande grupo, existem áreas menores, que parecem ter sido criados para classificar alguns  específicos.

#### Variação nos dados vs predição para instâncias individuais

Para observar o quanto a variação nos dados pode impactar a predição para instâncias individuais, vamos fazer um último experimento. Execute o código abaixo e observe a saída do modelo para cada instância previamente selecionada (conforme definidas no código). As funções abaixo propositalmente não utilizam um random_state fixo.

**Responda >>>** Informe as classes preditas pelo modelo para as cinco instâncias selecionadas neste [link](https://docs.google.com/spreadsheets/d/1QMOWestK5ZUCbsDDuBgBCxVCStW2qXMyydKnYkgftMM/edit?usp=sharing).




In [ ]:
X_interesting = X[[40, 86, 297, 135, 73], :]

## Instanciar uma nova árvore de decisão, dessa vez sem especificar o estado
dt = DecisionTreeClassifier()

## Separar o conjunto em treino e teste, dessa vez sem especificar o estado
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,stratify=y)

## Treinar a nova árvore usando o conjunto de treino
dt.fit(X_train, y_train)

## Usar a nova árvore treinada para obter predições para os valores de X acima.
y_pred = dt.predict(X_interesting)
print(y_pred)

### Estratégias de podas e seus efeitos

As árvores de decisão treinadas nos itens anteriores não possuíam nenhuma forma de poda. Por padrão, a função disponibilizada pelo scikit-learn **não restringe o treinamento** das árvores.

No entanto, é possível utilizar técnicas de poda através do scikit-learn. Vamos analisar duas estratégias conforme discutido em aula: pré-poda e pós-poda.

Os exercícios abaixo visam demonstrar o efeito da poda na estrutura da árvore e na fronteira de decisão.

In [ ]:
## Separar o conjunto em treino e teste para os experimentos seguintes
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0,stratify=y)

#### Pré-poda: exemplos
Podemos especificar a profundidade máxima da árvore utilizando o hiperparâmetro `max_depth`. Iniciamos testando max_depth=2. Faça alterações no valor de `max_depth` para observar os resultados obtidos com diferentes profundidades máximas

In [ ]:
dt = DecisionTreeClassifier(max_depth=4)
dt.fit(X_train, y_train)

from sklearn.tree import plot_tree
plt.figure(figsize=(12,6))
_ = plot_tree(dt, feature_names=feature_names, class_names=target_names)

A célula de código abaixo analise o impacto de max_depth na fronteira de decisão. Novamente, simpĺificamos o conjunto de dados para fins de visualização: analisamos apenas dois atributos e um subconjunto de 50 instâncias.

In [ ]:
fig, axs = plt.subplots(2, 2, figsize = (15, 10),sharey=True, sharex=True)
plt.sca(axs[0,0])
plot_decision_boundaries(X_train[:50,:],y_train[:50],DecisionTreeClassifier, random_state=0)
plt.title("Sem restrição", fontsize=13)
plt.sca(axs[0,1])
plot_decision_boundaries(X_train[:50,:],y_train[:50],DecisionTreeClassifier, random_state=0,max_depth=3)
plt.title("max_depth=3", fontsize=13)
plt.sca(axs[1,0])
plot_decision_boundaries(X_train[:50,:],y_train[:50],DecisionTreeClassifier, random_state=0,max_depth=2)
plt.title("max_depth=2", fontsize=13)
plt.sca(axs[1,1])
plot_decision_boundaries(X_train[:50,:],y_train[:50],DecisionTreeClassifier, random_state=0,max_depth=1)
plt.title("max_depth=1", fontsize=13)
plt.show()

O método [DecisionTreeClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html) possui outros hiperparâmetros que podem ser ajustadas para um early *stopping* do treinamento da árvore de decisão: 


*   `min_samples_leaf`: *The minimum number of samples required to be at a leaf node. A split point at any depth will only be considered if it leaves at least min_samples_leaf training samples in each of the left and right branches. This may have the effect of smoothing the model, especially in regression.*
*   `min_samples_split`: *This may have the effect of smoothing the model, especially in regression.*
*   `max_leaf_nodes`: *Grow a tree with max_leaf_nodes in best-first fashion*


Vamos repetir a análise anterior, limitando o valor do hiperparâmetro `min_samples_leaf`. 


In [ ]:
#min_samples_leaf pode ser um valor inteiro ou um valor real: no segundo caso, é interpretado como uma proporção dos dados de treinamento
dt = DecisionTreeClassifier(min_samples_leaf=100,random_state=0) 
dt.fit(X_train, y_train)

from sklearn.tree import plot_tree
plt.figure(figsize=(12,6))
_ = plot_tree(dt, feature_names=feature_names, class_names=target_names)

In [ ]:
fig, axs = plt.subplots(ncols=3, figsize = (15, 4),sharey=True, sharex=True)
plt.sca(axs[0])
plot_decision_boundaries(X_train[:50,:],y_train[:50],DecisionTreeClassifier, random_state=0)
plt.title("Sem restrição", fontsize=13)
plt.sca(axs[1])
plot_decision_boundaries(X_train[:50,:],y_train[:50],DecisionTreeClassifier, random_state=0,min_samples_leaf=3)
plt.title("min_samples_leaf=3", fontsize=13)
plt.sca(axs[2])
plot_decision_boundaries(X_train[:50,:],y_train[:50],DecisionTreeClassifier, random_state=0,min_samples_leaf=5)
plt.title("min_samples_leaf=5", fontsize=13)
plt.show()

#### Pós-poda: Custo-complexidade

A biblioteca scikit-learn possui uma implementação de pós-poda por custo complexidade, baseada em custo-complexidade $\alpha \ge 0$.

Na implementação descrita na biblioteca, é definido também um custo-complexidade efetivo do nodo. Quanto maior for a taxa de erros ao se podar a subárvore de um nodo, maior será seu custo-complexidade efetivo. Além disso, quanto maior for a complexidade (número de nodos terminais) da subárvore do nodo, menor será seu custo-complexidade efeito.
Em resumo, um nodo com alto custo-complexidade efetivo é um nodo importante para diminuir a taxa de erros e com baixa complexidade.

Dentro da biblioteca, passamos um parâmetro $ccp\_alpha$ que serve como um custo-complexidade efetivo de corte: subárvores são podadas enquanto houver nodos com custo-complexidade menor do que o parâmetro $ccp\_alpha$.
Ou seja, quando maior for o parâmetro, mais intensa será a poda. 

Para mais informações:
* https://scikit-learn.org/stable/modules/tree.html#minimal-cost-complexity-pruning
* https://scikit-learn.org/stable/auto_examples/tree/plot_cost_complexity_pruning.html


In [ ]:
dt = DecisionTreeClassifier(random_state=0, ccp_alpha=0.001)
dt.fit(X_train, y_train)

from sklearn.tree import plot_tree
plt.figure(figsize=(12,6))
_ = plot_tree(dt, feature_names=feature_names, class_names=target_names)

In [ ]:
dt = DecisionTreeClassifier(random_state=0, ccp_alpha=0.05)
dt.fit(X_train, y_train)

from sklearn.tree import plot_tree
plt.figure(figsize=(12,6))
_ = plot_tree(dt, feature_names=feature_names, class_names=target_names)

In [ ]:
fig, axs = plt.subplots(ncols=2, figsize = (15, 4),sharey=True, sharex=True)
plt.sca(axs[0])
plot_decision_boundaries(X_train[:50,:],y_train[:50],DecisionTreeClassifier, random_state=0)
plt.title("Sem restrição", fontsize=13)
plt.sca(axs[1])
plot_decision_boundaries(X_train[:50,:],y_train[:50],DecisionTreeClassifier, random_state=0,ccp_alpha=0.05)
plt.title("ccp_alpha=0.05", fontsize=13)
plt.show()


O ajuste de `ccp_alpha`, bem como dos demais hiperparâmetros avaliados na pré-poda, não é algo trivial. Cada problema apresenta seu valor ótimo. No código abaixo vamos exemplificar como ajustar o valor de `ccp_alpha` para o problema em questão, testando diferentes possibilidades em um intervalo pré-definido.

Opcionalmente, o scikit-learn fornece o método `cost_complexity_pruning_path` que ajuda a estima os valores de ccp_alpha mais efetivos, reduzindo o número de valores a serem testados. (Veja o código de exemplo ao final deste notebook)

In [ ]:
## Definição da função para gerar um gráfico acurácia vs valores do hiperparâmetro
## hiperparâmetros escolhidos: 

def plot_acc_vs_hypervalues(accuracies_train, accuracies_test, hyper_values, hyper_name):
  fig, ax = plt.subplots(figsize=(12, 6))
  ax.set_xlabel(hyper_name)
  ax.set_ylabel("accuracy")
  ax.set_title("Accuracy vs " + hyper_name + " for training and testing sets")
  ax.plot(hyper_values, accuracies_train, marker="o", label="train", drawstyle="steps-post")
  ax.plot(hyper_values, accuracies_test, marker="o", label="test", drawstyle="steps-post")
  ax.legend()
  ax.grid()
  plt.show()

## para armazenar desempenhos em treino e teste
accs_train = []
accs_test = []

## definindo manualmente o intervalo de valores a ser testado para ccp
ccps = [k * 0.001 for k in range(0, 75, 3)]

for ccp in ccps:
  dt = DecisionTreeClassifier(ccp_alpha=ccp, random_state=0)
  dt.fit(X_train, y_train)
  
  y_pred_train = dt.predict(X_train)
  acc_train = accuracy_score(y_train, y_pred_train)

  y_pred_test = dt.predict(X_test)
  acc_test = accuracy_score(y_test, y_pred_test)

  accs_train.append(acc_train)
  accs_test.append(acc_test)

plot_acc_vs_hypervalues(accs_train, accs_test, hyper_values=ccps, hyper_name = "alpha")

**Responda >>>** De acordo com os resultados obtidos no gráfico acima, qual seria o melhor valor (ou melhores valores) de `ccp_alpha` a serem utilizados na poda da árvore para este conjunto de dados?

> ***Sua resposta aqui:*** De acordo com o gráfico os valores de ccp_alpha entre 0.0 e 0.01 apresentaram os melhores resultados.

---

## Sua vez

Seguindo o exemplo da otimização do valor do hiperparâmetro `ccp_alpha` para o treinamento de árvores de decisão para classificação do tipo de tumor, maligno ou benigno, faça a otimização de hiperparâmetros que controlam a liberdade de treinamento da árvore de decisão, isto é, que realizam **pré-poda**.
Varie os valores e avalie o efeito sobre o desempenho de pelo menos dois destes hiperparâmetros, conforme discutido na seção 'Pré-poda: exemplos'. 
Apresente como resultados da sua atividade:


*   O gráfico que compara o desempenho vs o valor do hiperparâmetro para os dados de treino e teste.
*   A escolha do melhor valor de cada hiperparâmetro analisado, de acordo com os gráficos gerados.
*   A matriz de confusão para os dados de teste dos modelos gerados com os valores de hiperparâmetros otimizados neste exercício, comparando-os também a uma árvore de decisão treinada com pós-poda (escolha o valor de `ccp_alpha` de acordo com a otimização do hiperparâmetro feita neste notebook). Reporte acurácia, precisão e recall. 
*   Qual abordagem foi melhor para treinar uma árvore de decisão que reduza os falsos negativos, isto é, maximize o recall?




In [ ]:
## Solução SUA Vez

## hiperparametro min_samples_leaf
accs_train = []
accs_test = []
min_samples_leafs = range(5, 30)

for min_samples_leaf in min_samples_leafs:
  dt = DecisionTreeClassifier(min_samples_leaf=min_samples_leaf, random_state=0)
  dt.fit(X_train, y_train)
  
  y_pred_train = dt.predict(X_train)
  acc_train = accuracy_score(y_train, y_pred_train)

  y_pred_test = dt.predict(X_test)
  acc_test = accuracy_score(y_test, y_pred_test)

  accs_train.append(acc_train)
  accs_test.append(acc_test)

plot_acc_vs_hypervalues(accs_train, accs_test, hyper_values=min_samples_leafs, hyper_name = "min samples leaf")

## hiperparametro max_leaf_node
accs_train = []
accs_test = []
max_leaf_nodes = range(5, 20)

for max_leaf_node in max_leaf_nodes:
  dt = DecisionTreeClassifier(max_leaf_nodes=max_leaf_node, random_state=0)
  dt.fit(X_train, y_train)
  
  y_pred_train = dt.predict(X_train)
  acc_train = accuracy_score(y_train, y_pred_train)

  y_pred_test = dt.predict(X_test)
  acc_test = accuracy_score(y_test, y_pred_test)

  accs_train.append(acc_train)
  accs_test.append(acc_test)

plot_acc_vs_hypervalues(accs_train, accs_test, hyper_values=max_leaf_nodes, hyper_name = "max leaf node")

# escolhendo os melhores parametros pré
print('Visualizando melhores hiperparâmetro pré')

x = 7 # min_samples_leaf
y = 15 # max_leaf_nodes 

dt_pre = DecisionTreeClassifier(min_samples_leaf=x, max_leaf_nodes=y, random_state=0)
dt_pre.fit(X_train, y_train)
y_pred_pre = dt_pre.predict(X_test)

cm = confusion_matrix(y_test, y_pred_pre,labels=dt_pre.classes_)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=dt_pre.classes_)
disp = disp.plot(include_values=True, cmap='Blues', ax=None, xticks_rotation='horizontal')
plt.grid(False)
plt.show()

print('Acurácia: {}'.format(round(accuracy_score(y_test, y_pred_pre),3)))
print('Recall: {}'.format(round(recall_score(y_test, y_pred_pre,pos_label=0),3)))
print('Precisão: {}'.format(round(precision_score(y_test, y_pred_pre,pos_label=0),3)))

# escolhendo os melhores parametros pós
print('Visualizando melhores hiperparâmetro Pós')
dt_pos = DecisionTreeClassifier(ccp_alpha=0.005, random_state=0)
dt_pos.fit(X_train, y_train)
y_pred_pos = dt_pos.predict(X_test)

cm = confusion_matrix(y_test, y_pred_pos,labels=dt_pos.classes_)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=dt_pos.classes_)
disp = disp.plot(include_values=True, cmap='Blues', ax=None, xticks_rotation='horizontal')
plt.grid(False)
plt.show()

print('Acurácia: {}'.format(round(accuracy_score(y_test, y_pred_pos),3)))
print('Recall: {}'.format(round(recall_score(y_test, y_pred_pos,pos_label=0),3)))
print('Precisão: {}'.format(round(precision_score(y_test, y_pred_pos,pos_label=0),3)))

# As duas abordagens apresentaram os mesmos melhores resultados para o recall, 0.952. 



---


#### Sugestões de experimentos extras (opcionais):

*   Expanda o seu estudo de pré-poda para os demais hiperparâmetros não explorados na atividade prática, fazendo a otimização dos seus valores. 
*   Analise o efeito no desempenho no modelo ao utilizar o hiperparâmetro `class_weigths='balanced'` no método DecisionTreeClassifier. Esta configuração do hiperparâmetro atributo usa os valores de $y$ para ajustar automaticamente pesos inversamente proporcionais às frequências de classe nos dados de entrada, tentando reduzir o impacto do desbalanceamento entre classes.
*    Faça experimentos utilizando `criterion='entropy'` na escolha dos melhores atributos para divisão de nós da árvore. 
*   Execute a otimização do hiperparâmetro `ccp_alpha` com os valores efetivos de `ccp_alpha` retornados pela função cost_complexity_pruning_path. Veja o exemplo utilizado na aula teórica [aqui](https://scikit-learn.org/stable/auto_examples/tree/plot_cost_complexity_pruning.html). Os códigos com uso da função cost_complexity_pruning_path são fornecidos abaixo.



In [ ]:
## estima os valores de alpha efetivos e as respectivas impurezas.
clf = DecisionTreeClassifier(random_state=0)
path = clf.cost_complexity_pruning_path(X_train, y_train)
ccp_alphas, impurities = path.ccp_alphas, path.impurities

In [ ]:
## Faz a análise visual da variação de impureza de acordo com o valor efetivo
fig, ax = plt.subplots()
ax.plot(ccp_alphas[:-1], impurities[:-1], marker="o", drawstyle="steps-post")
ax.set_xlabel("effective alpha")
ax.set_ylabel("total impurity of leaves")
ax.set_title("Total Impurity vs effective alpha for training set")

In [ ]:
## Treina diversas árvores de decisão, uma para cada valor efetivo de alpha
clfs = []
for ccp_alpha in ccp_alphas:
    clf = DecisionTreeClassifier(random_state=0, ccp_alpha=ccp_alpha)
    clf.fit(X_train, y_train)
    clfs.append(clf)
print(
    "Number of nodes in the last tree is: {} with ccp_alpha: {}".format(
        clfs[-1].tree_.node_count, ccp_alphas[-1]
    )
)

In [ ]:
## Analisa a complexidade da árvore de acordo com a variação do valor efetivo de alpha
## Retira o último valor de alpha do array, pois representa a poda mais drástica, que deixa
## apenas o nó folha
clfs = clfs[:-1]
ccp_alphas = ccp_alphas[:-1]

node_counts = [clf.tree_.node_count for clf in clfs]
depth = [clf.tree_.max_depth for clf in clfs]
fig, ax = plt.subplots(2, 1)
ax[0].plot(ccp_alphas, node_counts, marker="o", drawstyle="steps-post")
ax[0].set_xlabel("alpha")
ax[0].set_ylabel("number of nodes")
ax[0].set_title("Number of nodes vs alpha")
ax[1].plot(ccp_alphas, depth, marker="o", drawstyle="steps-post")
ax[1].set_xlabel("alpha")
ax[1].set_ylabel("depth of tree")
ax[1].set_title("Depth vs alpha")
fig.tight_layout()

In [ ]:
## Analisa a variação de desempenho para treino e teste.
train_scores = [clf.score(X_train, y_train) for clf in clfs]
test_scores = [clf.score(X_test, y_test) for clf in clfs]

fig, ax = plt.subplots()
ax.set_xlabel("alpha")
ax.set_ylabel("accuracy")
ax.set_title("Accuracy vs alpha for training and testing sets")
ax.plot(ccp_alphas, train_scores, marker="o", label="train", drawstyle="steps-post")
ax.plot(ccp_alphas, test_scores, marker="o", label="test", drawstyle="steps-post")
ax.legend()
plt.show()